In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# About negative magnitudo
The magnitude is a numerical representation of the total energy released during an earthquake, calculated based on the amplitude of ground oscillations caused by the passage of seismic waves, recorded by seismometers (seismograms). Surprisingly to many, earthquakes with negative magnitude can also be recorded (similar to temperature, magnitude can be negative). This is because Charles Richter, the creator of the Richter scale, defined a magnitude value of 0 for an earthquake that produced a maximum amplitude of 0.001 mm on a standard seismograph placed 100 km away from the epicenter. He further established that every time the maximum amplitude recorded increased tenfold compared to the previous value, the magnitude value would increase by one unit.


In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import plotly.express as px
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("/kaggle/input/the-ultimate-earthquake-dataset-from-1990-2023/Eartquakes-1990-2023.csv")

In [ ]:
df.date = pd.to_datetime(df.date)

In [ ]:
d = df.set_index("date")

d["magnitudo"].plot(kind = "line", style = ".", title ="Magnitudo trend by year", figsize =(16,5))
plt.show()

In [ ]:
plt.figure(figsize = (16,7))
sns.histplot(df["magnitudo"], bins = 250)
plt.title("Magnitudo bimodal distribution")
plt.show()

In [ ]:
def filter_plot_day(df, date_1,date_2):
    """ Return a map plot filtered by date"""
    df.copy()
    df = df.loc[(df.date > date_1) & (df.date < date_2)]
    fig = px.density_mapbox(df, lat='latitude', lon='longitude', z='magnitudo', radius=10,
                        center=dict(lat=0, lon=180), zoom=0,
                        title =f"<br> Eartquake magnitudo beetween {date_1} and {date_2}")
    fig.update_layout(mapbox_style="open-street-map")
    fig.show()

filter_plot_day(df, date_1 = "2023-02-01", date_2 ="2023-02-02")

## Let's extrapolate some feature from the date

In [ ]:
def transform_date(df):
    df = df.copy()
    df["hour"] = df.date.dt.hour
    df["day_of_month"] = df.date.dt.day
    df["weekday"] = df.date.dt.day_name()
    df["month"] = df.date.dt.month
    df["month_name"] = df.date.dt.month_name()
    df["year"] =df.date.dt.year
    return df


In [ ]:
df = transform_date(df)

In [ ]:
plt.figure(figsize=(16,7))
sns.boxenplot(df,x="year", y="magnitudo",linewidth=1 )
plt.xticks(rotation=90)
plt.title("Eartquakes magnitudo distribution by Year")
plt.show()

In [ ]:
top_mag = df.sort_values(by="magnitudo", ascending = False).head(20)

plot = top_mag.sort_values(by="magnitudo").plot(x="place",y="magnitudo", kind ="barh",width=1, edgecolor="black",figsize =(8,7),color = "orange")
for bars in plot.containers:
        plot.bar_label(bars, padding=-45, fontsize=12, color="white", weight="bold")
plt.title("Strongest earthquakes recorded from 1990-2023")
plt.xlabel("Richter scale")
plt.show()

In [ ]:
def plot_strongest(df):
    gb_top = df.groupby(["state"])\
               .agg({"significance":"max", "magnitudo":"max"})\
               .reset_index()

    b = df[["year", "state","significance","longitude", "latitude","magnitudo"]]
    merge = pd.merge(gb_top, b, how = "left", on =["state","significance", "magnitudo"]).sort_values(by="year")


    fig = px.scatter_geo(merge, 
                         lat="latitude", lon="longitude", 
                         size="significance",
                         animation_frame="year",
                         projection="natural earth",
                         title = "Strongest eartquake by year")
    fig.show()
    
plot_strongest(df)